In [ ]:
import os
import re
import requests

# 创建保存图片的目录
output_dir = '/Users/huqianghui/Downloads/Multi-Modal-RAG/image_captions/images'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 读取文件
with open('/Users/huqianghui/Downloads/Multi-Modal-RAG/image_captions/img_files_1.txt', 'r') as file:
    lines = file.readlines()

# 正则表达式匹配imageUrl
url_pattern = re.compile(r"'imageUrl':\s*'([^']+)'")

# 下载并保存图片
for line in lines:
    match = url_pattern.search(line)
    if match:
        url = match.group(1)
        try:
            response = requests.get(url)
            response.raise_for_status()  # 检查请求是否成功
            # 从URL中提取文件名
            filename = os.path.join(output_dir, os.path.basename(url))
            # 保存图片
            with open(filename, 'wb') as img_file:
                img_file.write(response.content)
            print(f"Downloaded and saved {filename}")
        except requests.exceptions.RequestException as e:
            print(f"Failed to download {url}: {e}")

In [ ]:
from dataclasses import dataclass

@dataclass
class ImageData:
    id: str
    imageUrl: str
    caption: str

# List to store the objects
image_data_list = []

# Read the file
with open('/Users/huqianghui/Downloads/Multi-Modal-RAG/image_captions/img_files_1.txt', 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Manually parse the line to extract fields
            line = line.strip()
            id_start = line.find("'id': '") + len("'id': '")
            id_end = line.find("',", id_start)
            image_id = line[id_start:id_end]

            image_url_start = line.find("'imageUrl': '") + len("'imageUrl': '")
            image_url_end = line.find("',", image_url_start)
            image_url = line[image_url_start:image_url_end]

            caption_start = line.find("'caption': '") + len("'caption': '")
            caption_end = line.rfind("'}")
            caption = line[caption_start:caption_end]

            # Escape special characters
            image_id = image_id.replace("'", "\\'")
            image_url = image_url.replace("'", "\\'")
            caption = caption.replace("'", "\\'")

            # Create an ImageData object
            image_data = ImageData(id=image_id, imageUrl=image_url, caption=caption)
            # Add the object to the list
            image_data_list.append(image_data)
        except Exception as e:
            print(f"Error processing line: {line}")
            print(f"Error message: {e}")

# Print the list of objects
for item in image_data_list:
    print(item)

In [6]:
from openai import AzureOpenAI

api_base = "https://openai-hu-non-product-test.openai.azure.com/"
api_key= "41433befdf824b348136ef86bb897073"
deployment_name = "gpt-4o"
api_version = '2024-02-01' # this might change in the future

client = AzureOpenAI(
    api_key=api_key,  
    api_version=api_version,
    base_url=f"{api_base}/openai/deployments/{deployment_name}"
)

response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        { "role": "system", "content": "You are an AI assistant that helps people find information and description information from the upload pictures in Chinese. Just response the related content. Do not say anything unrelated." },
        { "role": "user", "content": [  
            { 
                "type": "text", 
                "text": "Describe this picture:" 
            },
            { 
                "type": "image_url",
                "image_url": {
                    "url": "https://img2.tapimg.com/moment/etag/lhZEbeJKeI5qOwQxlRSUTsZcYen0.png"
                }
            }
        ] } 
    ],
    max_tokens=500 
)

print(response.choices[0].message.content)

这是一张游戏截图，界面左侧有多个功能按钮，包括“Tap社区”、“公告”、“好友”、“设置”、“小狗游记”、“开服狂欢”、“解药师”、“大玩家”、“开服特惠”、“首充礼包”、“旅行活动”等。屏幕中央和右侧显示了游戏中的场景，包含一些帐篷、营火、一些NPC和摆摊的区域。右上角显示了游戏内的货币数量：1,066颗蓝色星星和447k金币。整个场景充满了卡通和奇幻风格的元素。屏幕底部有五个按钮，分别是：“营地”、“行李”、“冒险”、“旅人”和“麦乐兽”。
